<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classficiation-CNN/Faster_R_CNN_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Required Libraries

In [35]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import xml.etree.ElementTree as ET
import albumentations as A
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

from google.colab import drive

Define File paths

In [36]:
# Mount Google Drive
drive.mount('/content/drive')

# Define dataset paths
DATASET_PATH = "/content/drive/MyDrive/DSGP/Preprocessed_Dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "Train")
TEST_PATH = os.path.join(DATASET_PATH, "Test")
VAL_PATH = os.path.join(DATASET_PATH, "Val")

# Check dataset structure
if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"Train folder not found: {TRAIN_PATH}")
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"Test folder not found: {TEST_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define Custom Faster R-CNN Model

In [37]:
# Define Faster R-CNN model from scratch
class CustomFasterRCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomFasterRCNN, self).__init__()

        # Use a ResNet50 backbone without pretrained weights
        backbone = resnet_fpn_backbone('resnet50', pretrained=False)

        # Define the model
        self.model = FasterRCNN(
            backbone,
            num_classes=num_classes,
            rpn_anchor_generator=AnchorGenerator(
                sizes=((32, 64, 128, 256, 512),),
                aspect_ratios=((0.5, 1.0, 2.0),) * 5
            )
        )

    def forward(self, images, targets=None):
        return self.model(images, targets)

Define Dataset Class

In [38]:
# Custom Dataset Class
class TumorDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.image_files = []
        self.annotation_files = []
        self.labels_set = set()

        images_path = os.path.join(root_dir, "Images")
        annotations_path = os.path.join(root_dir, "Annotations")

        # Ensure folders exist
        if not os.path.exists(images_path):
            raise FileNotFoundError(f"Missing 'Images' directory in {root_dir}")
        if not os.path.exists(annotations_path):
            raise FileNotFoundError(f"Missing 'Annotations' directory in {root_dir}")

        for tumor_type in sorted(os.listdir(images_path)):
            tumor_images_dir = os.path.join(images_path, tumor_type)
            tumor_annotations_dir = os.path.join(annotations_path, tumor_type)

            if not os.path.isdir(tumor_images_dir) or not os.path.isdir(tumor_annotations_dir):
                continue

            # Sort files to maintain order
            tumor_images = sorted(os.listdir(tumor_images_dir))
            tumor_annotations = sorted(os.listdir(tumor_annotations_dir))

            for image in tumor_images:
                image_path = os.path.join(tumor_images_dir, image)
                annotation_path = os.path.join(tumor_annotations_dir, os.path.splitext(image)[0] + ".xml")

                if os.path.exists(annotation_path):
                    self.image_files.append(image_path)
                    self.annotation_files.append(annotation_path)

                    # Extract labels from annotation
                    _, labels = self.parse_annotation(annotation_path)
                    self.labels_set.update(labels)

        # Create a fixed label mapping
        self.label_dict = {name: idx + 1 for idx, name in enumerate(sorted(self.labels_set))}

    def __len__(self):
        return len(self.image_files)

    def parse_annotation(self, annotation_path):
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []

        for obj in root.findall("object"):
            name = obj.find("name").text
            labels.append(name)  # Tumor class name

            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            boxes.append([xmin, ymin, xmax, ymax])

        return np.array(boxes, dtype=np.float32), labels

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        annotation_path = self.annotation_files[idx]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        boxes, labels = self.parse_annotation(annotation_path)

        labels = [self.label_dict[label] for label in labels]


        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": labels
        }

        if self.transforms:
            label_strings = [str(label) for label in labels]
            sample = self.transforms(image=image, bboxes=boxes, class_labels=labels)
            image = sample["image"]
            target["boxes"] = torch.tensor(sample["bboxes"], dtype=torch.float32)

        return image, target

Define Data Transformations

In [39]:
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.pytorch.transforms.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))


Load dataset and DataLoader

In [40]:
train_dataset = TumorDataset(TRAIN_PATH, transforms=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

Initialize Model and Training Setup

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get number of tumor classes
unique_classes = set()
for annotation in train_dataset.annotation_files:
    tree = ET.parse(annotation)
    root = tree.getroot()
    for obj in root.findall("object"):
        unique_classes.add(obj.find("name").text)

num_classes = len(unique_classes) + 1  # Tumor classes + background

# Initialize custom Faster R-CNN model
model = CustomFasterRCNN(num_classes=num_classes)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

Train the Model

In [43]:
num_epochs = 1
losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, targets in train_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) if isinstance(v, list) else v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    losses.append(total_loss)
    print(f"Epoch {epoch+1}, Loss: {total_loss}")

AssertionError: Anchors should be Tuple[Tuple[int]] because each feature map could potentially have different sizes and aspect ratios. There needs to be a match between the number of feature maps passed and the number of sizes / aspect ratios specified.

Plot Training Loss

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), losses, marker='o', linestyle='-', color='b')
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss Over Epochs")
plt.show()

Save the Model

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/DSGP/faster_rcnn_tumor_classification.pth")
print("Model saved!")

Evaluate the Model on Test Data

In [ ]:
model.eval()
image_path = "/content/drive/MyDrive/DSGP/CNN_dataset/Test/Images/astrocitoma/sample.jpg"

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_transformed = transform(image=image)["image"].unsqueeze(0).to(device)
output = model(image_transformed)

Compute the IoU Score

In [ ]:
# Function to Calculate IoU
def calculate_iou(boxA, boxB):
    """Calculate Intersection over Union (IoU)."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    intersection = max(0, xB - xA) * max(0, yB - yA)
    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = boxA_area + boxB_area - intersection

    return intersection / union if union > 0 else 0

# Compute IoU
ious = []
for i, target in enumerate(output[0]['boxes']):
    iou = calculate_iou(target.cpu().numpy(), train_dataset.parse_annotation(image_path)[0])
    ious.append(iou)

# Plot IoU Scores
plt.figure(figsize=(10, 5))
plt.hist(ious, bins=20, color='g', alpha=0.7)
plt.xlabel("IoU Score")
plt.ylabel("Frequency")
plt.title("IoU Score Distribution")
plt.show()